<a href="https://colab.research.google.com/github/nndkhoa/llm-local/blob/main/deepseek_r1_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!curl -fsSL https://ollama.com/install.sh | sh
!pip install -U langchain-ollama
!pip install colab-xterm
%load_ext colabxterm

In [ ]:
%xterm
# OLLAMA_HOST="0.0.0.0" OLLAMA_ORIGINS="*" ollama serve
# Chạy xong lệnh trên thì để yên nó ở đây

In [ ]:
%xterm
# ollama pull deepseek-r1 & ollama pull nomic-embed-text
# Xong có thể clear output cho gọn

In [ ]:
from langchain_ollama.llms import OllamaLLM
llm = OllamaLLM(model="deepseek-r1")

In [ ]:
# Chỉ tạo để test ollama đang chạy
response = llm.invoke("who are your?")

print(response)

<think>

</think>

Greetings! I'm DeepSeek-R1, an artificial intelligence assistant created by DeepSeek. I'm at your service and would be delighted to assist you with any inquiries or tasks you may have.


In [ ]:
# Mở thêm một tab nữa để cung cấp api
%xterm

# ssh -p 443 -R0:localhost:11434 qr@a.pinggy.io

# Test cục bộ với
# curl http://localhost:11434/api/generate -d '{ "model": "deepseek-r1", "prompt": "Who are you?", "stream": false}'



In [ ]:
# Test với local địa chỉ sinh ra, ví dụ: http://rncli-35-203-177-46.a.free.pinggy.link

# curl  http://rnikh-35-197-47-109.a.free.pinggy.link/api/generate -d '{ "model": "deepseek-r1", "prompt": "Who are you?", "stream": false}'

In [ ]:
%xterm

# curl http://localhost:11434/api/generate -d '{ "model": "deepseek-r1", "prompt": "Who are you?", "stream": false}'

In [ ]:
!pip install gradio ollama chromadb langchain-community langchain-core

In [ ]:
import ollama
from bs4 import BeautifulSoup as bs
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import Chroma
from langchain_community.embeddings import OllamaEmbeddings

# Load the data from the web URL
url = 'https://vnexpress.net/de-xuat-thoi-gian-lam-viec-du-ngan-de-nam-nu-co-thoi-gian-tim-ban-doi-4778404.html'
loader = WebBaseLoader(url)
docs = loader.load()

# or load the text from pdf
# path = "../data/Understanding_Climate_Change.pdf"
# loader = PyPDFLoader(path)
# documents = loader.load()

# Split the loaded documents into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

# Create Ollama embeddings and vector store
embeddings = OllamaEmbeddings(model="nomic-embed-text") # or use (model = "llama3")
vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings)

# Define the function to call the Ollama Llama3 model
def ollama_llm(question, context):
    formatted_prompt = f"Question: {question}\n\nContext: {context}"
    response = ollama.chat(model='deepseek-r1', messages=[{'role': 'user', 'content': formatted_prompt}])
    return response['message']['content']

# Define the RAG setup
retriever = vectorstore.as_retriever() # as_retriever(search_kwargs={"k": 2})

def rag_chain(question):
    retrieved_docs = retriever.invoke(question)
    formatted_context = "\n\n".join(doc.page_content for doc in retrieved_docs)
    return ollama_llm(question, formatted_context)





In [ ]:
# Giao diện nhập liệu form bên trong Colab
import gradio as gr
iface = gr.Interface(
  fn=rag_chain,
  inputs=gr.Textbox(lines=2, placeholder="Enter your question here..."),
  outputs="text",
  title="RAG with Llama3.1",
  description="Ask questions about the provided context",
)

# Launch the Gradio app
iface.launch()

In [ ]:
print(rag_chain("Tóm tắt nội dung bài viết"))

<think>
Okay, I need to summarize the content of this Vietnamese article. The article starts by talking about Vietnamese laws limiting couples to having 1-2 children, which has been in place for several years. Each age group has different policies, and currently, penalties for having a third child are applied. However, these penalties don't seem appropriate anymore because Vietnam is facing a population decline that's below replacement level but could become a major issue without intervention.

The article mentions that the Vietnamese government will continue to review its population policies to make them better suited to current realities in the near future, aiming for the best demographic structure and quality in the long run. There's also a forecast of the law from the Prime Minister scheduled for December 2025.

Looking at the additional content section, there's an article about suggesting the optimal time frame for working so that men and women can have enough time to find their l